In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL da página
url = "https://fbref.com/pt/comps/22/cronograma/Major-League-Soccer-Resultados-e-Calendarios"

# Faz a requisição para a página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Função para extrair a tabela da página
def extrair_tabela(rows):
    headers = []
    data = []
    
    for row in rows:
        cols = row.find_all(['th', 'td'])
        if len(headers) == 0:  # A primeira linha será o cabeçalho
            headers = [col.get_text(strip=True) for col in cols]
        else:  # Demais linhas são dados
            data.append([col.get_text(strip=True) for col in cols])
    
    # Cria o DataFrame
    return pd.DataFrame(data, columns=headers)

# Encontra o container da tabela
table_container = soup.find('div', {'class': 'table_container'})
rows = table_container.find_all('tr')

# Extrai a tabela
tabela = extrair_tabela(rows)

# Encontra a coluna "Relatório da Partida"
links_relatorios = []
for row in rows:
    a_tag = row.find('a', text='Relatório da Partida')
    if a_tag:
        href = a_tag.get('href')
        if href and href != "#":  # Ignora links vazios ou "#" como marcador
            links_relatorios.append(href)
        else:
            links_relatorios.append(None)
    else:
        links_relatorios.append(None)

# Remove linhas onde a coluna "Relatório da Partida" é "Confronto"
tabela = tabela[~tabela['Relatório da Partida'].str.contains("Confronto", na=False)]

# Adiciona a coluna de links na tabela
tabela['Link Relatório'] = links_relatorios[:len(tabela)]  # Ajusta o tamanho da lista para o tamanho da tabela

# Exibe a tabela final com os links
print("\nTabela com links:")
print(tabela.head())

# Se precisar dos links, aqui está a lista filtrada
print("\nLinks 'Relatório da Partida':")
print(tabela['Link Relatório'].dropna().tolist())



Tabela com links:
  Sem  Dia        Data Horário        Em casa   xG Resultado   xG  \
0   1  qua  2024-02-21   20:00    Inter Miami  1.4       2–0  0.8   
1   1  sáb  2024-02-24   13:45           LAFC  1.5       2–1  1.8   
2   1  sáb  2024-02-24   14:00           Crew  1.8       1–0  0.5   
3   1  dom  2024-02-25   14:30  FC Cincinnati  0.9       0–0  0.5   
4   1  dom  2024-02-25   16:00      Nashville  0.1       0–0  1.4   

      Visitante Público            Local Árbitro  Relatório da Partida Notas  \
0           RSL  21.137    Chase Stadium          Relatório da Partida         
1       Seattle  22.214      BMO Stadium          Relatório da Partida         
2   Atlanta Utd  20.406  Lower.com Field          Relatório da Partida         
3    Toronto FC  25.513      TQL Stadium          Relatório da Partida         
4  NY Red Bulls  30.109      Geodis Park          Relatório da Partida         

                                      Link Relatório  
0                             

C:\Users\eduar\AppData\Local\Temp\ipykernel_11112\1845203675.py:37: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = row.find('a', text='Relatório da Partida')


In [2]:
# Desloca a coluna "Link Relatório" uma linha para cima
tabela['Link Relatório'] = tabela['Link Relatório'].shift(-1)

# Preenche a última linha da coluna "Link Relatório" com NaN (caso não esteja automaticamente)
tabela.loc[tabela.index[-1], 'Link Relatório'] = None


In [3]:
# Remove linhas onde "Link Relatório" é None
tabela = tabela[tabela['Link Relatório'].notna()]

In [4]:
tabela

,Sem,Dia,Data,Horário,Em casa,xG,Resultado,xG,Visitante,Público,Local,Árbitro,Relatório da Partida,Notas,Link Relatório
0,1,qua,2024-02-21,20:00,Inter Miami,1.4,2–0,0.8,RSL,21.137,Chase Stadium,,Relatório da Partida,,/pt/partidas/6b8eefb5/Inter-Miami-Real-Salt-La...
1,1,sáb,2024-02-24,13:45,LAFC,1.5,2–1,1.8,Seattle,22.214,BMO Stadium,,Relatório da Partida,,/pt/partidas/2d02571f/Los-Angeles-FC-Seattle-S...
2,1,sáb,2024-02-24,14:00,Crew,1.8,1–0,0.5,Atlanta Utd,20.406,Lower.com Field,,Relatório da Partida,,/pt/partidas/50d19f80/Columbus-Crew-Atlanta-Un...
3,1,dom,2024-02-25,14:30,FC Cincinnati,0.9,0–0,0.5,Toronto FC,25.513,TQL Stadium,,Relatório da Partida,,/pt/partidas/7add352c/FC-Cincinnati-Toronto-FC...
4,1,dom,2024-02-25,16:00,Nashville,0.1,0–0,1.4,NY Red Bulls,30.109,Geodis Park,,Relatório da Partida,,/pt/partidas/58d9874f/Nashville-SC-New-York-Re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,31,sáb,2024-09-14,19:30,St. Louis,1.6,1–3,2.3,Minnesota Utd,,Citypark,Marcos de Oliveira,Relatório da Partida,,/pt/partidas/7d556c1d/St-Louis-City-Minnesota-...
438,31,sáb,2024-09-14,19:30,Vancouver W'caps,2.8,2–0,1.0,SJ Earthquakes,,BC Place Stadium,Lukasz Szpala,Relatório da Partida,,/pt/partidas/a3e99752/Vancouver-Whitecaps-FC-S...
439,31,sáb,2024-09-14,19:30,Dynamo FC,1.5,4–1,0.8,RSL,,Shell Energy Stadium,Jon Freemon,Relatório da Partida,,/pt/partidas/e0a8e36f/Houston-Dynamo-Real-Salt...
440,31,sáb,2024-09-14,19:30,LA Galaxy,2.4,4–2,1.6,LAFC,,Dignity Health Sports Park,Ted Unkel,Relatório da Partida,,/pt/partidas/e7b27721/El-Trafico-LA-Galaxy-Los...


In [7]:
tabela.to_csv('C://Users//eduar//Desktop//DEV//Football//MLS//links.csv', index=False)